In [ ]:
# 11/18/2021-1
from __future__ import absolute_import, division
from IPython.display import clear_output
from BashColors import C
from CV2_Utils import *
import cv2, glob, os
from os.path import *
import tensorflow as tf
from tensorflow import keras

contentPath = os.getcwd()
dataPath = join(contentPath, 'data')
testPath = join(contentPath, 'TestImages')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
base_model = keras.applications.MobileNetV3Small(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
cv2Path=join(contentPath, 'CV2Images')
testPath=join(contentPath, 'TestImages')

from tensorflow.keras.utils import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=4
)
print()

validation_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=4)

print()
# clear_output()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 2
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)
# clear_output()

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
# model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history=model.fit(train_ds,
                  epochs=epochs,
                  validation_data=validation_ds)

In [ ]:
modelName = base_model.name + '_retrained.h5'
savePath = join(contentPath, modelName)
print(savePath)
model.save(savePath)

In [ ]:
new_model = tf.keras.models.load_model(savePath)
# new_model.summary() # Check its architecture
inputShape=new_model.input_shape
print(inputShape)
new_model.build(input_shape=inputShape)

In [ ]:
os.chdir(testPath)
testGlobList=[]
testGlobList=glob.glob('**')
os.chdir(contentPath)

for pth in testGlobList:
    fullPath=abspath(pth)
    img=cv2.imread(fullPath)
    print(type(img))
    sleep(0.05)
    img=None

In [ ]:
import numpy as np
from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img(fullPath, target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

model.predict(img)